In [1]:
!pip install fastapi uvicorn nest_asyncio pydantic


In [2]:
import nest_asyncio
nest_asyncio.apply()


In [7]:
!pip install PyPDF2
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import io
import PyPDF2

from google.colab import userdata
import google.generativeai as genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.0 MB/s eta 0:00:00


In [3]:
# ================================
# Enterprise Agent – Colab Version
# ================================

import uuid, time, json, logging
from typing import Dict, Any, List
from pydantic import BaseModel

# ---------- Structured Logging ----------
logger = logging.getLogger("enterprise_agent")
logger.handlers.clear()
handler = logging.StreamHandler()
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.setLevel(logging.INFO)
logger.addHandler(handler)

def log_trace(trace: Dict[str, Any]):
    logger.info(json.dumps(trace))


# ---------- In-memory Session Memory ----------
SESSIONS = {}  # store chat history + embeddings (mock)

def get_session(session_id=None):
    if not session_id:
        session_id = str(uuid.uuid4())
    if session_id not in SESSIONS:
        SESSIONS[session_id] = {
            "history": [],
            "created": time.time()
        }
    return session_id, SESSIONS[session_id]


# ---------- Mock LLM ----------
def llm_generate(prompt: str):
    return "AI Response: " + prompt[:200]


# ---------- Planner ----------
def planner(user_text: str):
    steps = []
    if "summarize" in user_text.lower():
        steps.append({"action": "search_docs", "args": {"query": user_text}})
        steps.append({"action": "summarize", "args": {}})
    else:
        steps.append({"action": "web_search", "args": {"query": user_text}})
    return steps


# ---------- Tools ----------
def tool_web_search(query: str):
    return f"[MOCK WEB] Results for: {query}"

def tool_doc_search(query: str):
    return [
        f"[MOCK DOC] Paragraph about {query}",
        f"[MOCK DOC] Additional info about {query}"
    ]

def tool_summarize(snippets: List[str]):
    return " ".join(snippet[:120] for snippet in snippets)


TOOLS = {
    "web_search": tool_web_search,
    "search_docs": tool_doc_search,
    "summarize": tool_summarize,
}


# ---------- Agent Executor ----------
def run_agent(user_prompt: str, session_id=None, user_id="test_user"):

    session_id, sess = get_session(session_id)

    trace = {
        "trace_id": str(uuid.uuid4()),
        "session_id": session_id,
        "user_id": user_id,
        "start": time.time(),
        "steps": []
    }

    sess["history"].append({"from": "user", "text": user_prompt})

    plan = planner(user_prompt)
    trace["plan"] = plan
    tool_outputs = {}

    for step_no, step in enumerate(plan):
        action = step["action"]
        args = step["args"]
        step_trace = {
            "step_no": step_no + 1,
            "action": action,
            "start": time.time(),
            "args": args
        }

        try:
            if action == "web_search":
                out = tool_web_search(args["query"])
            elif action == "search_docs":
                out = tool_doc_search(args["query"])
            elif action == "summarize":
                out = tool_summarize(tool_outputs.get("search_docs", []))
            else:
                out = "Unknown tool"

            tool_outputs[action] = out
            step_trace["status"] = "ok"
            step_trace["result"] = str(out)[:200]

        except Exception as e:
            step_trace["status"] = "error"
            step_trace["error"] = str(e)

        step_trace["end"] = time.time()
        trace["steps"].append(step_trace)
        log_trace(trace)

    # Final LLM response
    final_output = llm_generate(user_prompt + " | tool_outputs=" + str(tool_outputs))
    sess["history"].append({"from": "agent", "text": final_output})

    trace["final"] = final_output[:200]
    trace["end"] = time.time()
    log_trace(trace)

    return {
        "session_id": session_id,
        "output": final_output,
        "trace_id": trace["trace_id"]
    }


In [4]:
run_agent("Summarize the information about enterprise agent architecture")


{"trace_id": "dfbdd5e9-64ce-4cf0-bd95-45fbac64ff2d", "session_id": "89fd6201-4076-4bfc-b69d-036249b910cd", "user_id": "test_user", "start": 1763184828.5671344, "steps": [{"step_no": 1, "action": "search_docs", "start": 1763184828.567144, "args": {"query": "Summarize the information about enterprise agent architecture"}, "status": "ok", "result": "['[MOCK DOC] Paragraph about Summarize the information about enterprise agent architecture', '[MOCK DOC] Additional info about Summarize the information about enterprise agent architecture']", "end": 1763184828.567152}], "plan": [{"action": "search_docs", "args": {"query": "Summarize the information about enterprise agent architecture"}}, {"action": "summarize", "args": {}}]}
INFO:enterprise_agent:{"trace_id": "dfbdd5e9-64ce-4cf0-bd95-45fbac64ff2d", "session_id": "89fd6201-4076-4bfc-b69d-036249b910cd", "user_id": "test_user", "start": 1763184828.5671344, "steps": [{"step_no": 1, "action": "search_docs", "start": 1763184828.567144, "args": {"qu

{'session_id': '89fd6201-4076-4bfc-b69d-036249b910cd',
 'output': "AI Response: Summarize the information about enterprise agent architecture | tool_outputs={'search_docs': ['[MOCK DOC] Paragraph about Summarize the information about enterprise agent architecture', '[MOCK DOC] Ad",
 'trace_id': 'dfbdd5e9-64ce-4cf0-bd95-45fbac64ff2d'}

In [5]:
SESSIONS


{'89fd6201-4076-4bfc-b69d-036249b910cd': {'history': [{'from': 'user',
    'text': 'Summarize the information about enterprise agent architecture'},
   {'from': 'agent',
    'text': "AI Response: Summarize the information about enterprise agent architecture | tool_outputs={'search_docs': ['[MOCK DOC] Paragraph about Summarize the information about enterprise agent architecture', '[MOCK DOC] Ad"}],
  'created': 1763184828.5671184}}